In [1]:
import os

import sqlite3
import pandas as pd

In [2]:
# Path to the root directory of datasci
dthon_path = os.path.join(os.path.expanduser('~'), 'Projects','datasci')
dthon_db = os.path.join(dthon_path, "melbourne-datathon","data","medi.db")
# Run this first to create database connection
conn = sqlite3.connect(dthon_db)

In [4]:
# Anthonies query
sql = """
SELECT t.Prescription_Week, COUNT(Prescriber_ID) as c
        FROM transactions t
        JOIN ChronicIllness_LookUp l ON t.Drug_ID = l.MasterProductID
        JOIN patients p on p.Patient_ID = t.Patient_ID 
        WHERE l.ChronicIllness = 'Depression'
        AND p.postcode LIKE '3%'
        GROUP BY t.Prescription_Week
        ORDER BY t.Prescription_Week
        LIMIT 10
"""

%time df = pd.read_sql_query(sql, conn)
df.head()

CPU times: user 27.8 s, sys: 26.6 s, total: 54.4 s
Wall time: 4min 39s


,Prescription_Week,c
0,2009-08-30,1
1,2009-12-06,1
2,2010-01-03,5
3,2010-01-10,5
4,2010-01-17,14


In [6]:
# Anthonies query adam fix
sql = """
SELECT t.Prescription_Week, COUNT(Prescriber_ID) as c
        FROM transactions t
        JOIN ChronicIllness_LookUp l ON t.Drug_ID = l.MasterProductID
        WHERE l.ChronicIllness = 'Depression'
        AND Patient_ID IN (
            SELECT Patient_ID FROM patients
            WHERE postcode LIKE '3%'
        )
        GROUP BY t.Prescription_Week
        ORDER BY t.Prescription_Week
        LIMIT 10
"""

%time df = pd.read_sql_query(sql, conn)
df.head()

CPU times: user 22.1 s, sys: 26.2 s, total: 48.3 s
Wall time: 4min 46s


,Prescription_Week,c
0,2009-08-30,1
1,2009-12-06,1
2,2010-01-03,5
3,2010-01-10,5
4,2010-01-17,14


In [35]:
sql = """
    SELECT MasterProductID, count(*) as c FROM Drug_LookUp
    GROUP BY MasterProductID
    HAVING c != 1
"""

df = pd.read_sql_query(sql, conn)
df.head()

,MasterProductID,c


In [11]:
sql = """
    SELECT * FROM ChronicIllness_LookUp;
"""

df = pd.read_sql_query(sql, conn)
df.head()

,ChronicIllness,MasterProductID,MasterProductFullName
0,ChronicIllness,MasterProductID,MasterProductFullName
1,Diabetes,141,ACTOS TAB 30MG (AS HCL) 28
2,Diabetes,142,ACTOS TAB 45MG (AS HCL) 28
3,Osteoporosis,146,ACTONEL ONCE-A-MONTH TAB 150MG 1
4,Osteoporosis,267,ALENDROBELL TAB 70MG 4


In [40]:
# How many drugs treat multiple diseases
sql = """
    SELECT ci.ChronicIllness, ci.MasterProductID
    FROM ChronicIllness_LookUp ci
    WHERE ci.MasterProductID IN (
        SELECT MasterProductID
        FROM ChronicIllness_LookUp ci2
        WHERE ci2.ChronicIllness != ci.ChronicIllness);
"""

pd.read_sql_query(sql, conn)

,ChronicIllness,MasterProductID


In [4]:
# Any Drug transactions not from a chronic illness?
sql = """
    SELECT Drug_ID
    FROM transactions
    WHERE Drug_ID NOT IN (
        SELECT MasterProductID
        FROM ChronicIllness_LookUp
        GROUP BY MasterProductID
    )
"""
%time df_tc = pd.read_sql_query(sql, conn)

CPU times: user 1min, sys: 5.89 s, total: 1min 6s
Wall time: 1min 43s


In [50]:
# Any Drug transactions not from a chronic illness?
sql = """
    SELECT MasterProductID
    FROM ChronicIllness_LookUp
    WHERE MasterProductID NOT IN (
        SELECT Drug_ID
        FROM transactions
        GROUP BY Drug_ID
    )
    GROUP BY MasterProductID
"""
%time df_ct = pd.read_sql_query(sql, conn)

CPU times: user 14.8 s, sys: 579 ms, total: 15.4 s
Wall time: 15.6 s


In [5]:
df_tc

,Drug_ID
0,13
1,13
2,13
3,13
4,13
5,13
6,13
7,13
8,13
9,13


In [47]:
df_ct

,MasterProductID
0,146
1,273
2,297
3,428
4,429
5,551
6,579
7,705
8,771
9,773
